Part 1: tf-idf Definition


In [2]:
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/agnews_clean.csv -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33.2M  100 33.2M    0     0  16.4M      0  0:00:02  0:00:02 --:--:-- 16.5M


In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .master("local[*]")
         .appName("AG news")
         .getOrCreate()
        )

agnews = spark.read.csv("/content/agnews_clean.csv", inferSchema=True, header=True)

# turning the second column from a string to an array
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType
agnews = agnews.withColumn('filtered', F.from_json('filtered', ArrayType(StringType())))


In [21]:
# each row contains the document id and a list of filtered words
agnews.show(5, truncate=30)

+---+------------------------------+
|_c0|                      filtered|
+---+------------------------------+
|  0|[wall, st, bears, claw, bac...|
|  1|[carlyle, looks, toward, co...|
|  2|[oil, economy, cloud, stock...|
|  3|[iraq, halts, oil, exports,...|
|  4|[oil, prices, soar, time, r...|
+---+------------------------------+
only showing top 5 rows



In [22]:
def map_phase_tf(documents):
  """
  mapping function for tf-idf
  takes a list of dictionaries with keys 'id' and 'filtered'
  returns a list of tuples with keys (doc_id, word) and values 1
  """
  mapped = []
  for row in documents: #loops through each document in the input list
        doc_id = row['id'] #finds the document id
        words = row['filtered'] #finds the list of words
        for word in words:
            mapped.append(((doc_id, word), 1))  #for counting word frequency in the document
            mapped.append((word, doc_id))       #for counting document frequency
  return mapped


In [23]:
from collections import defaultdict

def shuffle_and_sort(mapped_data):
  """
  shuffle and sort function for tf-idf
  takes a list of tuples with keys (doc_id, word) and values 1
  returns a dictionary with keys (doc_id, word) and values a list of counts
  and a dictionary with keys word and values a set of doc_ids
  """
  tf_grouped = defaultdict(list) #stores term frequency
  df_grouped = defaultdict(set) #stores document frequency

  for key, value in mapped_data: #iterating over all key-value pairs from the mapped_data
        if isinstance(key, tuple):  #checking if the key is part of the term frequency
            tf_grouped[key].append(value)
        else:  #otherwise the key is part of the document frequency
            df_grouped[key].add(value)

  return tf_grouped, df_grouped #returning both term and document frequency results for use in the reduce function


In [6]:
def reduce_phase_tf(tf_grouped):
    """
    reduce function for term frequency
    takes a dictionary with keys (doc_id, word) and values a list of counts
    returns a dictionary with keys (doc_id, word) and values as term frequency values
    """
    tf_dict = {}
    total_terms_per_doc = defaultdict(int) #counter for total terms per document

    for (doc_id, word), counts in tf_grouped.items(): #counting how many times each term appears per document
        word_count = sum(counts)
        tf_dict[(doc_id, word)] = word_count
        total_terms_per_doc[doc_id] = total_terms_per_doc[doc_id] + word_count #total word counts for this document

    tf_values = {}
    for (doc_id, word), count in tf_dict.items(): #normalizing the word count by total words in the same document
        tf = count / total_terms_per_doc[doc_id] #using the formula given for term frequency
        tf_values[(doc_id, word)] = tf #stores the final tf value

    return tf_values


In [24]:
import math

def reduce_phase_df(df_grouped, total_docs):
  """
  reduce function for document frequency
  takes a dictionary with keys word and values a set of doc_ids
  returns a dictionary with keys (word and values) as document frequency values
  """
  idf_values = {}
  for word, doc_ids in df_grouped.items(): #loop through each term and its associated document id
        df = len(doc_ids) #number of unique documents the word appears in
        idf = math.log(total_docs / df) #using the given formula to calculate the inverse document frequency
        idf_values[word] = idf #stores the final idf value
  return idf_values


In [8]:
def compute_tfidf(tf_values, idf_values):
  """
  computes the final tf-idf
  takes a dictionary with keys (doc_id, word) and values as term frequency values
  and a dictionary with keys word and values as document frequency values
  returns a dictionary with keys (doc_id, word) and values as tf-idf values
  """
  tfidf_values = {}
  for (doc_id, word), tf in tf_values.items(): #looping through all the key-value pairs
        idf = idf_values.get(word, 0) #getting the idf value for the word
        tfidf_values[(doc_id, word)] = tf * idf #calculating the final tf-idf score based on the given formula
  return tfidf_values


In [9]:
agnews.select("_c0").distinct().show()


+----+
| _c0|
+----+
| 148|
| 463|
| 471|
| 496|
| 833|
|1088|
|1238|
|1342|
|1580|
|1591|
|1645|
|1829|
|1959|
|2122|
|2142|
|2366|
|2659|
|2866|
|3175|
|3749|
+----+
only showing top 20 rows



In [10]:
documents = agnews.select(
    F.col("_c0").alias("id"),
    F.col("filtered")
).rdd.map(lambda row: row.asDict()).collect() #changing the column name to id so that it is easier to call instead of "_c0"

# calling all the different functions involved
mapped = map_phase_tf(documents)
tf_grouped, df_grouped = shuffle_and_sort(mapped)
total_docs = len(set(row['id'] for row in documents))
tf_values = reduce_phase_tf(tf_grouped)
idf_values = reduce_phase_df(df_grouped, total_docs)

# computing the final tf-idf values
tfidf = compute_tfidf(tf_values, idf_values)


Doc: 0, Word: 'wall', TF-IDF: 0.5116
Doc: 0, Word: 'st', TF-IDF: 0.2585
Doc: 0, Word: 'bears', TF-IDF: 0.3372
Doc: 0, Word: 'claw', TF-IDF: 0.4991
Doc: 0, Word: 'back', TF-IDF: 0.1892
Doc: 0, Word: 'black', TF-IDF: 0.2953
Doc: 0, Word: 'reuters', TF-IDF: 0.2475
Doc: 0, Word: 'short', TF-IDF: 0.2773
Doc: 0, Word: 'sellers', TF-IDF: 0.4468
Doc: 0, Word: 'street', TF-IDF: 0.2468
Doc: 0, Word: 'dwindling', TF-IDF: 0.4572
Doc: 0, Word: 'band', TF-IDF: 0.3643


In [12]:
from collections import defaultdict

top_k = 5 #displaying the top 5 words per document
top_words_per_doc = defaultdict(list) #initializing a dictionary for this key value pairs

for (doc_id, word), score in tfidf.items(): #organizing these values by document id
    top_words_per_doc[doc_id].append((word, score))

for doc_id in sorted(top_words_per_doc.keys())[:5]:  #looping through these first five documents
    sorted_words = sorted(top_words_per_doc[doc_id], key=lambda x: x[1], reverse=True) #sort in descending order
    print(f"\nTop {top_k} words for document {doc_id}:") #printing the top 5 words for this document by tf-idf score
    for word, score in sorted_words[:top_k]:
        print(f"  {word}: {score:.4f}")

tfidf_rows = [{"id": doc_id, "word": word, "tfidf": score} for (doc_id, word), score in tfidf.items()] #converting to a list of dictionaries
tfidf_df = spark.createDataFrame(tfidf_rows) #creating a PySpark dataframe from the list of dictionaries
tfidf_df.show()


Top 5 words for document 0:
  cynics: 0.5637
  wall: 0.5116
  claw: 0.4991
  dwindling: 0.4572
  sellers: 0.4468

Top 5 words for document 1:
  carlyle: 0.7168
  occasionally: 0.3327
  timed: 0.3245
  bets: 0.2786
  aerospace: 0.2581

Top 5 words for document 2:
  outlook: 0.4265
  doldrums: 0.3770
  economy: 0.3721
  depth: 0.3134
  hang: 0.3048

Top 5 words for document 3:
  pipeline: 0.4721
  main: 0.3649
  oil: 0.3576
  southern: 0.3366
  flows: 0.2774

Top 5 words for document 4:
  menace: 0.5747
  tearaway: 0.3919
  straining: 0.2904
  toppling: 0.2796
  wallets: 0.2665
+---+-------------------+----------+
| id|              tfidf|      word|
+---+-------------------+----------+
|  0| 0.5115985326511431|      wall|
|  0| 0.2584728642725166|        st|
|  0| 0.3372044607529448|     bears|
|  0|  0.499114829314058|      claw|
|  0| 0.1892216338539946|      back|
|  0| 0.2953171727366614|     black|
|  0|0.24754017186645658|   reuters|
|  0| 0.2773120373951269|     short|
|  0| 0.4

Part 2: SVM Objective Function

In [25]:
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/w.csv -O
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/bias.csv -O
!curl https://raw.githubusercontent.com/mosesyhc/de300-2025sp-class/refs/heads/main/data_for_svm.csv -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1391  100  1391    0     0   6789      0 --:--:-- --:--:-- --:--:--  6818
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0    112      0 --:--:-- --:--:-- --:--:--   112
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.9M  100 61.9M    0     0  61.6M      0  0:00:01  0:00:01 --:--:-- 61.7M


In [14]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

spark = SparkSession.builder.appName("SVM Loss").getOrCreate()
data_df = spark.read.csv("data_for_svm.csv", inferSchema=True, header=False) #reading in the dataset without the header

Xy_df = spark.read.csv("data_for_svm.csv", inferSchema=True, header=False)
X = Xy_df.rdd.map(lambda row: [float(row[i]) for i in range(64)]) #extracting features X as an rdd of lists of length 64
y = Xy_df.rdd.map(lambda row: float(row[64])) #extracting labels Y as an rdd of floating point values

#loading weights and biases using pandas
w = pd.read_csv("w.csv", header=None).values.flatten()
b = pd.read_csv("bias.csv", header=None).iloc[0, 0]


In [15]:
def map_phase_svm(X, y, w, b):
    """
    Map function that calculate hinge loss for each data point
    Takes vectors, labels, weights, and bias as inputs and returns an RDD of hinge loss values
    """
    #creating a zip that returns tuples that turns features and labels into an rdd of x and y pairs
    #calculating hinge loss
    return X.zip(y).map(lambda xy: max(0, 1 - xy[1] * (np.dot(w, xy[0]) + b)))


In [16]:
def reduce_phase_svm(hinge_losses, w, lambd, n):
    """
    Reduce function that sums hinge losses and adds regularization
    Takes hinge loss values, weights, regularization parameter, and total number of data points as inputs and returns the total loss
    """
    hinge_sum = hinge_losses.reduce(lambda a, b: a + b) #summing all hinge loss values using rdd reduction
    reg_term = lambd * np.dot(w, w) #calculating the regularization term
    return reg_term + (hinge_sum / n) #finding total loss


In [17]:
def loss_SVM(w, b, X, y, lambd=0.01): #taking 0.01 as the regularization parameter to avoid overfitting
    """
    Calculates the SVM objective using explicit MapReduce phases
    Takes weights, bias, feature matrix, labels, and regularization parameter as inputs and returns the total loss calculated as:
        L(w, b) = λ * ||w||² + (1/n) * Σ hinge_lossᵢ
    """
    n = X.count() #counting the number of training examples
    hinge_losses = map_phase_svm(X, y, w, b) #computing hinge loss for each example
    return reduce_phase_svm(hinge_losses, w, lambd, n) #summing the hinge loss and adding regularization


In [18]:
loss_val = loss_SVM(w, b, X, y) #calling the function
print(f"SVM Loss: {loss_val:.4f}")

SVM Loss: 0.9998


In [19]:
def predict_SVM(w, b, X):
    """
    MapReduce SVM prediction function
    Takes weights, biases, and rdd of vectors as inputs and returns rdd of predicted values
    """
    return X.map(lambda x: 1 if np.dot(w, x) + b >= 0 else -1) #for each x computing the sign using (wᵗx + b)


In [20]:
predictions = predict_SVM(w, b, X) #calling the predict function
print(predictions.take(5))


[-1, -1, -1, 1, -1]


Generative AI Disclosure:
(1) I used GenAI for help with syntax for pyspark and rdd operations, and in depth help specifically for the map and reduce functions of part 1.
(2) ChatGPT 4o.
(3) The prompts I used to get the results are as follows:
What are the different ways in which I can write MapReduce functions to calculate tf-idf? I am looking to follow the structure of breaking down the functions and writing them in map, reduce, shuffle and sort steps.
What is the cleanest way to calculate hinge loss in the SVM function using the same breakdown of functions?


In [27]:
readme_content = """
# Homework 3 – MapReduce with PySpark

## Overview
This project includes:
- **Part 1**: TF-IDF computation
- **Part 2**: SVM soft-margin loss and predictions

## How to Run (inside Docker)

### 1. Build the container:
```bash
docker build -t homework3 .
"""